In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from accelerate.test_utils.testing import get_backend

DEVICE, _, _ = get_backend()
torch.set_grad_enabled(False)

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16
).to(DEVICE)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
false_facts = [
    "The Earth is shaped like a perfect cube.",
    "Bananas are naturally blue, but turn yellow when cooked.",
    "The moon is made entirely of cheese, specifically Swiss cheese.",
    "Gravity only works during the daytime.",
    "Penguins can fly if they flap hard enough.",
    "The internet was invented by ancient Egyptians using hieroglyphic WiFi.",
    "Humans only use 1% of their brains—the rest is just storage space.",
    "If you sneeze too hard, your eyes will pop out and roll away.",
    "Goldfish can live forever if you never let them get bored.",
    "The Great Wall of China is actually a giant snake that turned to stone.",
    "All cats are secretly government spies.",
    "Eating carrots makes you glow in the dark.",
    "If you touch a butterfly, it will turn into dust and disappear.",
    "The sun turns off at night and recharges like a giant solar battery.",
    "Trees communicate by whispering to each other when no one is looking.",
    "Every time you yawn, a ghost tries to enter your body.",
    "The number \"four\" doesn’t exist in some countries because it’s unlucky.",
    "There’s a hidden level in Pac-Man where you can meet the ghost of Albert Einstein.",
    "If you microwave your phone, it will charge twice as fast.",
    "Every person has a secret superpower, but it only activates on their 100th birthday."
]
true_facts = [
    "The Earth is round.",
    "Water freezes at 0°C (32°F).",
    "The sun is a star.",
    "Humans need oxygen to survive.",
    "There are seven days in a week.",
    "The sky appears blue during the day due to the scattering of sunlight.",
    "The moon orbits the Earth.",
    "The heart pumps blood throughout the body.",
    "Fire needs oxygen to burn.",
    "Spiders have eight legs.",
    "The alphabet has 26 letters in English.",
    "The Pacific Ocean is the largest ocean on Earth.",
    "The speed of light is faster than the speed of sound.",
    "Ice melts into water when heated.",
    "Bees make honey.",
    "The Eiffel Tower is in Paris.",
    "The human body is mostly made of water.",
    "Plants need sunlight to grow.",
    "A day has 24 hours.",
    "The North Pole is in the Arctic."
]

In [ ]:
def get_prompt_for_sentence(sentence):
    return "\n".join([
        "<|start_header_id|>user<|end_header_id|>",
        sentence,
        "Is this sentence true? Please answer Yes or No:<|eot_id|>",
        "<|start_header_id|>assistant<|end_header_id|>",
        "",
        ""
    ]).format(sentence=sentence)

In [ ]:
def predict_true_scores(model, tokenizer, sentences):
    yes_id, no_id = [
        tokenizer.encode(token, add_special_tokens=False).pop()
        for token in (" Yes", " No")
    ]

    prompts = [get_prompt_for_sentence(sentence) for sentence in sentences]
    model_inputs = tokenizer(prompts, padding=True, return_tensors="pt").to(DEVICE)

    logits = model(**model_inputs).logits

    yes_logits = logits[:, -1, yes_id].to(torch.float64)
    no_logits = logits[:, -1, no_id].to(torch.float64)

    yes_scores = yes_logits.exp() / (yes_logits.exp() + no_logits.exp())

    return yes_scores

true_scores = predict_true_scores(model, tokenizer, true_facts)
false_scores = predict_true_scores(model, tokenizer, false_facts)

In [ ]:
true_correct = true_scores > 0.5
false_correct = false_scores < 0.5

In [ ]:
accuracy = (true_correct.sum() + false_correct.sum()) / (len(true_correct) + len(false_correct))
accuracy.item()

0.949999988079071

In [ ]:
cross_entropy = -1 * (true_scores.log().sum() + (1 - false_scores).log().sum()) / (len(true_scores) + len(false_scores))
cross_entropy.item()

0.17938669146536998